## Simple usage example

In [ ]:
#Libraries
import json
import requests
import pandas as pd
import functions as F

In [ ]:
#Some constants
API_URL = "http://localhost:8000/"
CSV_FILE_PATH_EMPLOYEE = "./data/hired_employees.csv" 
CSV_FILE_PATH_DEPARTMENTS = "./data/departments.csv" 
CSV_FILE_PATH_JOBS = "./data/jobs.csv" 
RDS_SECRETARN = "rds!db-6849e7f9-0614-42df-8224-eeee9f554f17"


In [ ]:
# First Metric example
response = requests.get(f"{API_URL}metrics/quarterhires")
pd.DataFrame(json.loads(response.text)['data'])

In [ ]:
# Second Metric example
response = requests.get(f"{API_URL}metrics/topdepartments")
pd.DataFrame(json.loads(response.text)['data'])

In [ ]:
#Inyect employee data through the API
files = {'file': open(CSV_FILE_PATH_EMPLOYEE,'rb')}
response = requests.post(f"{API_URL}employees/upload", files=files)
print(response.status_code)
display(pd.DataFrame(json.loads(response.text)['data_inserted']))
display(pd.DataFrame(json.loads(response.text)['data_discarded']))


In [ ]:
#Inyect departments data through the API
files = {'file': open(CSV_FILE_PATH_DEPARTMENTS,'rb')}
response = requests.post(f"{API_URL}employees/upload", files=files)
pd.DataFrame(json.loads(response.text)['data_inserted'])

In [ ]:
#Inyect emplyee data through the API
files = {'file': open(CSV_FILE_PATH_JOBS,'rb')}
response = requests.post(f"{API_URL}employees/upload", files=files)
pd.DataFrame(json.loads(response.text)['data_inserted'])

### Just for development using jumbox/bastion to connect with RDS

In [1]:
import boto3
import json
import csv
import codecs
import functions as F
import pandas as pd
from mysql.connector import connect

In [2]:
ENV_VARIABLES = json.load(open('env_variables.json'))
BOTO3_SESSION = boto3.Session(profile_name="PERSONAL")


In [3]:

# Read Schemas
DB_NAME = ENV_VARIABLES["DB_NAME"]
DB_CREDENTIALS = F.get_secret(BOTO3_SESSION, 'us-east-1', ENV_VARIABLES["SECRET_RDS"])


In [19]:

# Consulta de inserción SQL

conn = connect(
    host="localhost",
    user=DB_CREDENTIALS['username'],
    password=DB_CREDENTIALS['password']
)

cursor = conn.cursor()
cursor.execute(F"USE {DB_NAME}")



In [23]:
# Inserta los registros en la base de datos utilizando executemany
cursor.execute("SELECT * FROM employees")

In [10]:
results = cursor.fetchall()
result_dicts = [dict(zip(cursor.column_names, row)) for row in results]
result_dicts

[]

In [24]:
conn.commit()
cursor.close()
conn.close()
